# XGBoost Model
##### Authors: Diego Senso González, Luis Vaciero
##### 4 december 2020
##### Module: Machine Learning - Master's Degree Data Science for Finance

## Libraries

We load the different libraries. Some of them will be used for the study in this document.

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
from scipy import stats as sts
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from math import sqrt
import itertools
import math
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve
from sklearn.datasets import make_classification
import random
random.seed(1234)
from sklearn.impute import KNNImputer
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

We load the data once we have cleaned it in previous steps.

In [95]:
#Loading data
loan = pd.read_csv("../notebooks/loan_lasso.csv", delimiter = ',')
loan

,Unnamed: 0,loan_status,num_rev_tl_bal_gt_0,D,sec_app_chargeoff_within_12_mths,dti,F,sec_app_inq_last_6mths,mort_acc,B,...,sec_app_open_acc,open_acc,sec_app_mort_acc,sec_app_open_act_il,G,A,policy_code,num_rev_accts,E,num_bc_sats
0,8,1.0,9.0,0,0.0,16.99,0,0.0,2.0,0,...,0.0,20.0,0.0,0.0,0,1,1.0,32.0,0,9.0
1,10,1.0,2.0,0,0.0,6.07,0,0.0,0.0,0,...,0.0,4.0,0.0,0.0,0,0,1.0,3.0,1,3.0
2,24,1.0,11.0,0,0.0,13.12,0,0.0,4.0,0,...,0.0,19.0,0.0,0.0,0,1,1.0,15.0,0,12.0
3,42,1.0,7.0,0,0.0,10.11,0,0.0,0.0,0,...,0.0,15.0,0.0,0.0,0,0,1.0,17.0,0,7.0
4,91,1.0,1.0,0,0.0,10.86,0,0.0,1.0,0,...,0.0,4.0,0.0,0.0,0,0,1.0,6.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462220,421088,1.0,7.0,1,0.0,27.08,0,0.0,0.0,0,...,0.0,9.0,0.0,0.0,0,0,1.0,13.0,0,7.0
462221,421089,0.0,8.0,0,0.0,23.69,0,0.0,1.0,1,...,0.0,13.0,0.0,0.0,0,0,1.0,20.0,0,4.0
462222,421092,0.0,6.0,1,0.0,30.90,0,0.0,0.0,0,...,0.0,9.0,0.0,0.0,0,0,1.0,10.0,0,4.0
462223,421093,0.0,7.0,0,0.0,27.19,0,0.0,0.0,0,...,0.0,17.0,0.0,0.0,0,0,1.0,16.0,1,7.0


In [96]:
#We delete first column which does not useful values
loan = loan.drop(loan.columns[[0]], axis='columns')
loan

,loan_status,num_rev_tl_bal_gt_0,D,sec_app_chargeoff_within_12_mths,dti,F,sec_app_inq_last_6mths,mort_acc,B,tax_liens,...,sec_app_open_acc,open_acc,sec_app_mort_acc,sec_app_open_act_il,G,A,policy_code,num_rev_accts,E,num_bc_sats
0,1.0,9.0,0,0.0,16.99,0,0.0,2.0,0,0.0,...,0.0,20.0,0.0,0.0,0,1,1.0,32.0,0,9.0
1,1.0,2.0,0,0.0,6.07,0,0.0,0.0,0,0.0,...,0.0,4.0,0.0,0.0,0,0,1.0,3.0,1,3.0
2,1.0,11.0,0,0.0,13.12,0,0.0,4.0,0,0.0,...,0.0,19.0,0.0,0.0,0,1,1.0,15.0,0,12.0
3,1.0,7.0,0,0.0,10.11,0,0.0,0.0,0,0.0,...,0.0,15.0,0.0,0.0,0,0,1.0,17.0,0,7.0
4,1.0,1.0,0,0.0,10.86,0,0.0,1.0,0,0.0,...,0.0,4.0,0.0,0.0,0,0,1.0,6.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462220,1.0,7.0,1,0.0,27.08,0,0.0,0.0,0,0.0,...,0.0,9.0,0.0,0.0,0,0,1.0,13.0,0,7.0
462221,0.0,8.0,0,0.0,23.69,0,0.0,1.0,1,0.0,...,0.0,13.0,0.0,0.0,0,0,1.0,20.0,0,4.0
462222,0.0,6.0,1,0.0,30.90,0,0.0,0.0,0,0.0,...,0.0,9.0,0.0,0.0,0,0,1.0,10.0,0,4.0
462223,0.0,7.0,0,0.0,27.19,0,0.0,0.0,0,0.0,...,0.0,17.0,0.0,0.0,0,0,1.0,16.0,1,7.0


# XGBoost

XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way.

## Model

Now we separate the variable to predict from the variables which will be used to explain it.

In [97]:
# endógena

endogena = ['loan_status'] 

# exógenas

exogenas = list(set(list(loan.columns))-set(endogena))

X = loan[exogenas].values
Y = loan[endogena].values

We create training set and test set using SMOTE

In [98]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=40)
sm = SMOTE(random_state=40, sampling_strategy = 0.4)
X_train, Y_train = sm.fit_sample(X_train, Y_train)
print(X_train.shape); print(X_test.shape)

(387126, 23)
(92445, 23)


Now we see the proportion of values 0 and 1 inside the training test and test set. This will be useful because we will be able to see if dataset is unbalanced, and if there is a huge difference between test and train set.

In [99]:
y_train = pd.DataFrame(Y_train, columns = ['loan_status'])
y_test = pd.DataFrame(Y_test, columns = ['loan_status'])

In [100]:
y_train['loan_status'].value_counts(normalize=True)

1.0    0.714287
0.0    0.285713
Name: loan_status, dtype: float64

In [101]:
y_test['loan_status'].value_counts(normalize=True)

1.0    0.745881
0.0    0.254119
Name: loan_status, dtype: float64

### Fitting

Now we fit the XGBoost model, make predictions and check the accuracy of our model.

In [9]:
# Fit model XGBoost
model = XGBClassifier()
model.fit(X_train, y_train)

C:\Users\diego\anaconda3\envs\gradiente\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Make predictions

In [10]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [11]:
# Evaluate predictions and get accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.04%


We see that have obtainen 75.04% of accuracy in this model. It is not a bad result, but we will have to see if this is good predicting new datasets which we do not have yet. Using "pickle" we can save the model to do this later.

### Saving

In [58]:
#We import pickle to save the model and use it later with new data
import pickle
filename = 'xgboost.sav'
pickle.dump(model, open (filename, 'wb'))
model2 = pickle.load(open('xgboost.sav', 'rb'))
model2

LogisticRegression(class_weight='balanced', max_iter=500, penalty='l1',
                   random_state=40, solver='saga')

### Loading and using for new data

We load the model we saved and use it to predict the new dataset which we give to the model.

In [102]:
#Load model and obtain accuracy for new dataset
model = pickle.load(open('xgboost.sav', 'rb'))
result = model.score(X_test, Y_test)
print("Accuracy: %.2f%%" % (result * 100.0))

Accuracy: 75.04%
